In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 4}
----------
[[ 1362  3812]
 [    0 13996]]
----------
              precision    recall  f1-score   support

           0       1.00      0.26      0.42      5174
           1       0.79      1.00      0.88     13996

    accuracy                           0.80     19170
   macro avg       0.89      0.63      0.65     19170
weighted avg       0.84      0.80      0.76     19170

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.16605616, 0.17403519, 0.1924845 , 0.26329625, 0.24185359,
        0.29720378, 0.32862139, 0.31964529, 0.3316139 , 0.34108758,
        0.39344811, 0.4223696 , 0.43284297, 0.41888034, 0.54504097,
        0.1855036 , 0.23387432, 0.26329589, 0.30418599, 0.38895929,
        0.3829751 , 0.41638529, 0.4612658 , 0.51362467, 0.49168491,
        0.53756273, 0.58443689, 0.58094621, 0.57845318, 0.59740174]),
 'std_fit_time': array([0.02243924, 0.0074805 , 0.00199568, 0.00598419, 0.00149548,
        0.03191495, 0.00947428, 0.01146877, 0.01047218, 0.00299263,
        0.00149608, 0.04338372, 0.01097131, 0.01097047, 0.01645553,
        0.0079788 , 0.0054872 , 0.004987  , 0.0029918 , 0.01396239,
        0.01196861, 0.00149524, 0.02044582, 0.03390932, 0.00598431,
        0.00598347, 0.02892208, 0.00349092, 0.00099766, 0.02493346]),
 'mean_score_time': array([0.00997365, 0.00997376, 0.00997305, 0.01097035, 0.01047111,
        0.01047206, 0.00997424, 0.01097071, 0.01246667, 0.01